Code adapated from https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

In [2]:
import requests
import pandas as pd
from datetime import datetime
import time

In [7]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

<Response [200]>

In [37]:
# # note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
# auth = requests.auth.HTTPBasicAuth('fPcc4PaCv-PIfyJR7_syRQ', 'jSoX7iF9c9EKxUHw8A4A1DFRch5Y6Q')

# # here we pass our login method (password), username, and password
# data = {'grant_type': 'password',
#         'username': 'lollersauce914',
#         'password': 'kittiesarecute1!'}

# # setup our header info, which gives reddit a brief description of our app
# headers = {'User-Agent': 'MyBot/0.0.1'}

# # send our request for an OAuth token
# res = requests.post('https://www.reddit.com/api/v1/access_token',
#                     auth=auth, data=data, headers=headers)

# # convert response to JSON and pull access_token value
# TOKEN = res.json()['access_token']

# # add authorization to our headers dictionary
# headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

In [82]:
# # initialize dataframe and parameters for pulling data in loop
# data = pd.DataFrame()
# params = {'limit': 100}

# # loop through 10 times (returning 1K posts)
# for i in range(10):
#     # make request
#     res = requests.get("https://oauth.reddit.com/r/gme/new",
#                        headers=headers,
#                        params=params)

#     # get dataframe from response
#     new_df = df_from_response(res)
#     # take the final row (oldest entry)
#     row = new_df.iloc[len(new_df)-1]
#     # create fullname
#     fullname = row['kind'] + '_' + row['id']
#     # add/update fullname in params
#     params['after'] = fullname
    
#     # append new_df to data
#     data = data.append(new_df, ignore_index=True)

In [66]:
# data.head()
# row = data.iloc[1]
# # create fullname
# fullname = row['id']

# fullname

'ofndn2'

In [91]:
# print(f"https://oauth.reddit.com/r/gme/comments/{fullname}/")
# test = requests.get(f"https://oauth.reddit.com/r/gme/comments/{fullname}/",
#                        headers=headers)

# test.json()

https://oauth.reddit.com/r/gme/comments/t3_ofjwds/


{'message': 'Not Found', 'error': 404}

In [126]:
# link = "https://api.pushshift.io/reddit/search/submission/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before=1625115600&size=100"
# res = requests.get(link)



In [145]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']:
        try:
            df = df.append({
                'subreddit': post['subreddit'],
                'title': post['title'],
                'selftext': post['selftext'],
                'upvote_ratio': post['upvote_ratio'],
                'created_utc': datetime.fromtimestamp(post['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
                'utc_raw': post['created_utc'],
                'id': post['id'],
                'kind': 't3'
            }, ignore_index=True)
        #catch missing self_text
        except:
            df = df.append({
                'subreddit': post['subreddit'],
                'title': post['title'],
                'selftext': '',
                'upvote_ratio': post['upvote_ratio'],
                'created_utc': datetime.fromtimestamp(post['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
                'utc_raw': post['created_utc'],
                'id': post['id'],
                'kind': 't3'
            }, ignore_index=True)

    return df

In [146]:
# get all posts from June on back to Jan 1 2021
before = 1625115600
data = pd.DataFrame()
newest_month = 0

while before > 1609480800:
    # make request
    res = requests.get(f"https://api.pushshift.io/reddit/search/submission/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=100")
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        break
    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)

starting on month 06.
starting on month 06.
starting on month 06.
starting on month 06.
starting on month 06.
starting on month 06.


IndexError: single positional indexer is out-of-bounds

In [154]:
data.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_posts.csv")

In [5]:
# we use this function to convert responses to dataframes
def df_from_comment(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']:
        try:
            df = df.append({
                'subreddit': post['subreddit'],
                'parent': post['parent_id'],
                'body': post['body'],
                'created_utc': datetime.fromtimestamp(post['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
                'utc_raw': post['created_utc'],
                'id': post['id'],
                'kind': 't2'
            }, ignore_index=True)
        #catch missing body
        except:
            df = df.append({
                'subreddit': post['subreddit'],
                'parent': post['parent_id'],
                'body': '',
                'created_utc': datetime.fromtimestamp(post['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
                'utc_raw': post['created_utc'],
                'id': post['id'],
                'kind': 't2'
            }, ignore_index=True)

    return df

In [182]:
# get all posts from June on back to Jan 1 2021
# June
before = 1625115600
june_comments = pd.DataFrame()
newest_month = 0

while before > 1622523600:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        continue
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    
    # append new_df to data
    june_comments = june_comments.append(new_df, ignore_index=True)

starting on month 06.
starting on month 05.


In [187]:
# get all posts from June on back to Jan 1 2021
before = 1622523600
may_comments = pd.DataFrame()
newest_month = 0

while before > 1619845200:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        continue
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    
    # append new_df to data
    may_comments = may_comments.append(new_df, ignore_index=True)

starting on month 05.
starting on month 04.


In [11]:
# get all posts from June on back to Jan 1 2021
before = 1619845200
apr_comments = pd.DataFrame()
newest_month = 0

while before > 1617253200:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        continue
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    
    # append new_df to data
    apr_comments = apr_comments.append(new_df, ignore_index=True)

starting on month 04.
starting on month 03.


In [14]:
# get all posts from June on back to Jan 1 2021
before = 1617253200
mar_comments = pd.DataFrame()
newest_month = 0
tot_comments = 0
while before > 1614578400:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        continue
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    tot_comments += 500
    if tot_comments % 10000 == 0 and tot_comments > 0:
            print(f'{tot_comments} processed so far.')
     
    # append new_df to data
    mar_comments = mar_comments.append(new_df, ignore_index=True)

starting on month 03.
10000 processed so far.
20000 processed so far.
30000 processed so far.
40000 processed so far.
50000 processed so far.
60000 processed so far.
70000 processed so far.
80000 processed so far.
90000 processed so far.
100000 processed so far.
110000 processed so far.
120000 processed so far.
130000 processed so far.
140000 processed so far.
150000 processed so far.
160000 processed so far.
170000 processed so far.
180000 processed so far.
190000 processed so far.
200000 processed so far.
210000 processed so far.
220000 processed so far.
230000 processed so far.
240000 processed so far.
250000 processed so far.
260000 processed so far.
270000 processed so far.
280000 processed so far.
290000 processed so far.
300000 processed so far.
310000 processed so far.
320000 processed so far.
330000 processed so far.
340000 processed so far.
350000 processed so far.
360000 processed so far.
370000 processed so far.
380000 processed so far.
390000 processed so far.
400000 proce

3200000 processed so far.
3210000 processed so far.
3220000 processed so far.
3230000 processed so far.
3240000 processed so far.
3250000 processed so far.
3260000 processed so far.
3270000 processed so far.
3280000 processed so far.
3290000 processed so far.
3300000 processed so far.
3310000 processed so far.
3320000 processed so far.
3330000 processed so far.
3340000 processed so far.
3350000 processed so far.
3360000 processed so far.
3370000 processed so far.
3380000 processed so far.
3390000 processed so far.
3400000 processed so far.
3410000 processed so far.
3420000 processed so far.
3430000 processed so far.
3440000 processed so far.
3450000 processed so far.
3460000 processed so far.
3470000 processed so far.
3480000 processed so far.
3490000 processed so far.
3500000 processed so far.
3510000 processed so far.
3520000 processed so far.
3530000 processed so far.
3540000 processed so far.
3550000 processed so far.
3560000 processed so far.
3570000 processed so far.
3580000 proc

In [15]:
# get all posts from June on back to Jan 1 2021
before = 1614578400
feb_comments = pd.DataFrame()
newest_month = 0
tot_comments = 0
while before > 1612159200:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        continue
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    tot_comments += 500
    if tot_comments % 10000 == 0 and tot_comments > 0:
            print(f'{tot_comments} processed so far.')
     
    # append new_df to data
    feb_comments = feb_comments.append(new_df, ignore_index=True)

starting on month 02.
10000 processed so far.
20000 processed so far.
30000 processed so far.
40000 processed so far.
50000 processed so far.
60000 processed so far.
70000 processed so far.
80000 processed so far.
90000 processed so far.
100000 processed so far.
110000 processed so far.
120000 processed so far.
130000 processed so far.
140000 processed so far.
150000 processed so far.
160000 processed so far.
170000 processed so far.
180000 processed so far.
190000 processed so far.
200000 processed so far.
210000 processed so far.
220000 processed so far.
230000 processed so far.
240000 processed so far.
250000 processed so far.
260000 processed so far.
270000 processed so far.
280000 processed so far.
290000 processed so far.
300000 processed so far.
310000 processed so far.
320000 processed so far.
330000 processed so far.
340000 processed so far.
350000 processed so far.
360000 processed so far.
370000 processed so far.
380000 processed so far.
390000 processed so far.
400000 proce

In [23]:
# get all posts from June on back to Jan 1 2021
before = 1612159200
jan_comments = pd.DataFrame()
newest_month = 0
tot_comments = 0
while before > 1609480800:
    # make request
    time.sleep(3)
    res = requests.get(f"https://api.pushshift.io/reddit/search/comment/?subreddit=gme&sort=desc&sort_type=created_utc&after=1609480800&before={before}&size=500")
    if res.ok == False and res.status_code >= 500:
        continue
    if res.ok == False and res.status_code < 500:
        print("too many requests")
        break
    # check for empty set
    if res.text == '{\n    "data": []\n}':
        break
    # get dataframe from response
    new_df = df_from_comment(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # get earliest date
    before = int(row['utc_raw'])
    # show where we're at
    if datetime.fromtimestamp(before).strftime('%m') != newest_month:
        newest_month = datetime.fromtimestamp(before).strftime('%m')
        print(f"starting on month {datetime.fromtimestamp(before).strftime('%m')}.")
    tot_comments += 500
    if tot_comments % 10000 == 0 and tot_comments > 0:
            print(f'{tot_comments} processed so far.')
     
    # append new_df to data
    jan_comments = jan_comments.append(new_df, ignore_index=True)

starting on month 01.
10000 processed so far.
20000 processed so far.
30000 processed so far.
40000 processed so far.
50000 processed so far.
60000 processed so far.
70000 processed so far.
80000 processed so far.
90000 processed so far.
100000 processed so far.
110000 processed so far.
120000 processed so far.
130000 processed so far.
140000 processed so far.


In [25]:
#print(len(apr_comments))
# print(len(mar_comments))
# print(len(feb_comments))
print(len(jan_comments))

#june_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_jun.csv")
#may_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_may.csv")
#apr_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_apr.csv")
# mar_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_mar.csv")
# feb_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_feb.csv")
jan_comments.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_comments_jan.csv")

28705


In [20]:
feb_comments.head()

,body,created_utc,id,kind,parent,subreddit,utc_raw
0,"Saluti dall'Italia, fanculo SEC!",2021-02-26T23:08:57Z,goy55dc,t2,t1_govbwlx,GME,1.614403e+09
1,"At this rate, by Paddy's Day the stock will ha...",2021-02-26T23:08:55Z,goy5549,t2,t1_gox1aj7,GME,1.614403e+09
2,With the area code included! 😡🦍,2021-02-26T23:08:53Z,goy54vr,t2,t1_gowvasm,GME,1.614403e+09
3,Am retarded. Am hodl.,2021-02-26T23:08:39Z,goy53g1,t2,t3_lt9x3l,GME,1.614403e+09
4,I like to keep things simple:\n\n🦍 see stock. ...,2021-02-26T23:08:36Z,goy532o,t2,t3_lt6zku,GME,1.614403e+09


In [3]:
res = requests.get("https://api.polygon.io/v1/open-close/GME/2020-10-14?adjusted=true&apiKey=FkJ0c2CuFs7HDQoJTSBaPXJdh_Mc90tU")

In [43]:
def price_df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()
    result = res.json()
    df = df.append({
         'status': result['status'],
         'date': result['from'],
         'open': result['open'],
         'high': result['low'],
         'close': result['close'],
         'volume': result['volume'],
         'afterHours': result['afterHours'],
         'preMarket': result['preMarket']}, ignore_index=True)
    return df

            
key = "FkJ0c2CuFs7HDQoJTSBaPXJdh_Mc90tU"
prices = pd.DataFrame()

for month in ["01","02","03","04","05","06"]:
    if month in ["01","03","05",]:
        last = 31
    if month == "02":
        last = 28
    else:
        last = 30
    for day in range(1,last+1):
        time.sleep(60)
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)
    
        res = requests.get(f"https://api.polygon.io/v1/open-close/GME/2021-{month}-{day_str}?adjusted=true&apiKey={key}")
        
        
        if res.status_code == 404:
            # market was closed
            continue
        
        else: 
            temp_df = price_df_from_response(res)
            print(f"finished {temp_df['date']}")
            prices = prices.append(temp_df)

finished 0    2021-01-04
Name: date, dtype: object
finished 0    2021-01-05
Name: date, dtype: object
finished 0    2021-01-06
Name: date, dtype: object
finished 0    2021-01-07
Name: date, dtype: object
finished 0    2021-01-08
Name: date, dtype: object
finished 0    2021-01-11
Name: date, dtype: object
finished 0    2021-01-12
Name: date, dtype: object
finished 0    2021-01-13
Name: date, dtype: object
finished 0    2021-01-14
Name: date, dtype: object
finished 0    2021-01-15
Name: date, dtype: object
finished 0    2021-01-19
Name: date, dtype: object
finished 0    2021-01-20
Name: date, dtype: object
finished 0    2021-01-21
Name: date, dtype: object
finished 0    2021-01-22
Name: date, dtype: object
finished 0    2021-01-25
Name: date, dtype: object
finished 0    2021-01-26
Name: date, dtype: object
finished 0    2021-01-27
Name: date, dtype: object
finished 0    2021-01-28
Name: date, dtype: object
finished 0    2021-01-29
Name: date, dtype: object
finished 0    2021-02-01
Name: 

In [45]:
prices.to_csv(r"C:\Users\Daniel\Documents\NLP Class\Project\gme_prices.csv")